In [1]:
import os

# 현재 경로 확인
current_dir = os.getcwd()

# 1단계 상위 디렉토리 경로 생성
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))


os.chdir(parent_dir)

In [2]:
import numpy as np
import pandas as pd
import joblib
x_data = np.load('./data/active.npy') #10 
c_data = np.load('./data/reaction.npy') #9
s_data = np.load('./data/promoter.npy') # 13
 
### train_test_split을 활용해 데이터를 train,val,test로 해서 0.6,0.2,0.2로 나눠서 구분
# - random_state = 21로 설정
from sklearn.model_selection import train_test_split
x_train,x_test,c_train,c_test = train_test_split(x_data,c_data, random_state = 21,test_size = 0.4)
x_val,x_test,c_val,c_test = train_test_split(x_test,c_test,random_state = 21, test_size = 0.5)
# x_train,x_val,x_test = [np.log1p(x) for x in [x_train,x_val,x_test]]
# c_train,c_val,c_test = [np.log1p(x) for x in [c_train,c_val,c_test]]
from sklearn.preprocessing import StandardScaler,MaxAbsScaler
x_scaler = MaxAbsScaler()
c_scaler = StandardScaler()
# s_scaler = StandardScaler()
x_train = x_scaler.fit_transform(x_train)
c_train = c_scaler.fit_transform(c_train)
x_val,x_test = [x_scaler.transform(x) for x in [x_val,x_test]]
c_val,c_test = [c_scaler.transform(c) for c in [c_val,c_test]]
# import joblib 
joblib.dump(x_scaler,'./torch/absx_scaler.pkl')
joblib.dump(c_scaler,'./torch/stac_scaler.pkl')
import torch
x_train,x_val,x_test = [torch.tensor(x, dtype = torch.float32) for x in [x_train,x_val,x_test]]
c_train,c_val,c_test = [torch.tensor(c, dtype = torch.float32) for c in [c_train,c_val,c_test]]
from torch.utils.data import TensorDataset,DataLoader
train_data = [x_train,c_train]
val_data = [x_val,c_val]
test_data = [x_test,c_test]
train_data = TensorDataset(*train_data)
val_data = TensorDataset(*val_data)
test_data = TensorDataset(*test_data)
datas = [train_data,val_data,test_data]
train_loader,val_loader,test_loader = [DataLoader(x,batch_size = 64,shuffle=False) for x in datas]
import os
os.makedirs('torch',exist_ok = True)
torch.save(train_loader,"torch/train_loader_a_r.pt")
torch.save(val_loader,"torch/val_loader_a_r.pt")
torch.save(test_loader,"torch/test_loader_a_r.pt")